# Severo Ochoa SKA Open Science School

<HR size=1>

## The Virtual Observatory

Jesús Salgado - SKA Regional Centre Network Architect

In [ ]:
!pip install astropy astroquery pyVO numpy matplotlib pandas statistics

In [ ]:
from astropy.table import Table
import astropy.units as u
import pyvo as vo
import numpy as np
import matplotlib.pyplot as plt

There are many useful astronomical methods in astropy. For example to resolve the coordinates of a certain target name, Simbad could be used directly in the code and SkyCoord could be used to convert coordinates systems

In [ ]:
from astropy.coordinates import SkyCoord
from astroquery.simbad import Simbad
result_table = Simbad.query_object("Pleiades")
c = SkyCoord(result_table['RA'], result_table['DEC'], unit=(u.hourangle, u.deg), frame='icrs')
radius  = 1.0                     # Degrees
inp_ra  = str(c.ra.degree[0])     # Degrees
inp_dec = str(c.dec.degree[0])    # Degrees

Now, we can use pyVO TAPService to query the Gaia TAP Service on sources around the Pleiades position

In [ ]:
tap_service = vo.dal.TAPService("https://gea.esac.esa.int/tap-server/tap/")

query = f"SELECT source_id,ra,dec,pmra,pmdec,parallax FROM gaiadr3.gaia_source_lite \
WHERE 1=CONTAINS(POINT({inp_ra}, {inp_dec}),CIRCLE(ra, dec, {radius})) and parallax_over_error > 10"

results = tap_service.run_sync(query)
print(results)

We represent the proper motions in ra and dec to identify comoving objects

In [ ]:
plt.scatter(results['pmra'], results['pmdec'], s=1)
z_fac = 140   # Zoom-in factor
plt.xlim([-z_fac,z_fac])
plt.ylim([-z_fac,z_fac])

plt.show()

There is a second cluster of sources so we represent only the ones close to this pmra/pmdec position.
We could refine the limits in the next point

In [ ]:
plt.scatter(results['pmra'], results['pmdec'], s=1)
plt.xlim(18,22)
plt.ylim(-43,-48)

plt.show()

Using this info, we can refine the query to select the Pleiades sources

In [ ]:
query = f"SELECT source_id,ra,dec,pmra,pmdec,parallax FROM gaiadr3.gaia_source_lite \
WHERE 1=CONTAINS(POINT({inp_ra}, {inp_dec}),CIRCLE(ra, dec, {radius})) and parallax_over_error > 10 and \
pmra > 18 and pmra < 22 and pmdec > -48 and pmdec < -43"

results = tap_service.run_sync(query)
print(results)

We represent the parallax of tht selected sources

In [ ]:
plt.hist(results['parallax'],100)
plt.show()

We can refine the query excluding outliers (outside Pleiades cluster)

In [ ]:
query = f"SELECT source_id,ra,dec,pmra,pmdec,parallax FROM gaiadr3.gaia_source_lite \
WHERE 1=CONTAINS(POINT({inp_ra}, {inp_dec}),CIRCLE(ra, dec, {radius})) and parallax_over_error > 10 and \
pmra > 18 and pmra < 22 and pmdec > -48 and pmdec < -43 and \
parallax > 6.5 and parallax < 8"

results = tap_service.run_sync(query)
print(results)

In [ ]:
plt.hist(results['parallax'],100)
plt.show()

Now that we are happy with the selection, we can calculate the parallax

In [ ]:
from statistics import mean
list_avg = mean(results['parallax']) 
print(list_avg)